<a href="https://colab.research.google.com/github/chefdarek/DS-Unit-2-Regression-2/blob/master/module1-log-linear-regression/Caterpillar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error
from functools import reduce


In [3]:
!pip install category_encoders


In [ ]:
bill_of_materials = pd.read_csv('bill_of_materials.csv')
comp_adaptor = pd.read_csv('comp_adaptor.csv')
comp_boss = pd.read_csv('comp_boss.csv')
comp_elbow = pd.read_csv('comp_elbow.csv')
comp_float = pd.read_csv('comp_float.csv')
comp_hf1 = pd.read_csv('comp_hfl.csv')
comp_nut = pd.read_csv('comp_nut.csv')
comp_other = pd.read_csv('comp_other.csv')
comp_sleeve = pd.read_csv('comp_sleeve.csv')
comp_straight = pd.read_csv('comp_straight.csv')
comp_tee = pd.read_csv('comp_tee.csv')
comp_threaded = pd.read_csv('comp_threaded.csv')
components = pd.read_csv('components.csv')
specs = pd.read_csv('specs.csv')
tube_end_form = pd.read_csv('tube_end_form.csv')
type_component = pd.read_csv('type_component.csv')
type_connection = pd.read_csv('type_connection.csv')
type_end_form = pd.read_csv('type_end_form.csv')
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')
tube = pd.read_csv('tube.csv')

In [ ]:
all_df = [bill_of_materials, comp_adaptor, comp_boss,comp_elbow,
  comp_float,comp_hf1,comp_nut, comp_other, comp_sleeve,
  comp_straight, comp_tee, comp_threaded, components, 
  specs, tube_end_form, type_component, type_connection, type_end_form]

In [ ]:
comps = list(all_df[1:11])
df_merged = pd.concat(comps)
                                           
df_merged.info()

In [7]:
all_comps = components.merge(df_merged, on='component_type_id')
all_comps.describe(include="object")

,component_id_x,name,component_type_id,base_type,blind_hole,component_id_y,connection_type_id,connection_type_id_1,connection_type_id_2,corresponding_shell,coupling_class,end_form_id_1,end_form_id_2,groove,material,mj_class_code,mj_plug_class_code,orientation,outside_shape,part_name,plating,thread_size,type,unique_feature
count,59066,59025,59066,15376,529,59066,18201,553,553,36,36,577,577,54374,36,14983,5664,59066,15376,0.0,2536,1443,15376,57451
unique,1854,272,25,3,2,852,10,5,5,6,3,4,4,2,3,5,3,2,2,0.0,2,24,2,2
top,C-1233,FLANGE,CP-018,Saddle,No,C-1865,B-005,B-004,B-004,C-0857,SP-0098,A-007,A-007,No,SP-0095,MJ-003,MJ-005,Yes,Round,NaN,No,M10,Boss,No
freq,124,13967,15376,10044,506,1006,7440,264,216,6,18,336,264,35651,18,11704,2955,51712,14384,NaN,2418,253,12896,44404


In [8]:
all_comps.component_type_id.unique()

array(['OTHER', 'CP-024', 'CP-026', 'CP-028', 'CP-018', 'CP-001',
       'CP-008', 'CP-009', 'CP-002', 'CP-010', 'CP-021', 'CP-011',
       'CP-027', 'CP-003', 'CP-004', 'CP-005', 'CP-019', 'CP-025',
       'CP-006', 'CP-020', 'CP-012', 'CP-022', 'CP-007', 'CP-023',
       'CP-029'], dtype=object)

### There is a way these fit in the puzzle I just haven't figured it out yet........I did however after a very long time concatting and merging they do not fit to bill_of_materials this way....

In [9]:
bill_of_materials.component_id_1.unique()

array(['C-1622', 'C-1312', 'C-1624', ..., 'C-0467', 'C-1694', 'C-1692'],
      dtype=object)

In [10]:
bill_of_materials.component_id_1.value_counts()

C-1621    2043
C-1622    1833
C-1624    1481
C-1623    1414
C-1312    1240
C-1625     950
C-1845     907
C-1620     676
C-1243     620
C-1420     469
C-0448     436
C-0444     219
C-0823     183
C-1229     155
C-1642     155
C-1638     154
C-1639     142
C-1848     138
C-1641     134
C-1626     134
C-1640     127
C-0101     120
C-1629     118
C-1631     117
C-1385      98
C-1637      81
C-1647      80
C-1649      80
C-1313      80
C-1369      78
          ... 
C-0892       1
C-1246       1
C-1413       1
C-1210       1
C-0443       1
C-1137       1
C-0838       1
C-1714       1
C-0889       1
C-0845       1
C-0572       1
C-1266       1
C-1665       1
C-0482       1
C-1280       1
C-0811       1
C-1027       1
C-2014       1
C-0649       1
C-0343       1
C-0024       1
C-1501       1
C-0905       1
C-0282       1
C-0330       1
C-0922       1
C-0189       1
C-1389       1
C-1455       1
C-0112       1
Name: component_id_1, Length: 1079, dtype: int64

In [11]:

for _ in all_df:
  print(_.columns.tolist())
  print(_.info())
  print(_.isna().sum())
  print(_.dtypes)
  print(_.shape)
  print(_.describe())
  print(_.describe(include="object"))

['tube_assembly_id', 'component_id_1', 'quantity_1', 'component_id_2', 'quantity_2', 'component_id_3', 'quantity_3', 'component_id_4', 'quantity_4', 'component_id_5', 'quantity_5', 'component_id_6', 'quantity_6', 'component_id_7', 'quantity_7', 'component_id_8', 'quantity_8']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21198 entries, 0 to 21197
Data columns (total 17 columns):
tube_assembly_id    21198 non-null object
component_id_1      19149 non-null object
quantity_1          19149 non-null float64
component_id_2      14786 non-null object
quantity_2          14786 non-null float64
component_id_3      4791 non-null object
quantity_3          4798 non-null float64
component_id_4      607 non-null object
quantity_4          608 non-null float64
component_id_5      92 non-null object
quantity_5          92 non-null float64
component_id_6      26 non-null object
quantity_6          26 non-null float64
component_id_7      7 non-null object
quantity_7          7 non-null float64
com

In [12]:
bill_of_materials.head()

,tube_assembly_id,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8
0,TA-00001,C-1622,2.0,C-1629,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00003,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00004,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00005,C-1624,1.0,C-1631,1.0,C-1641,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
train.head()

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,cost
0,TA-00002,S-0066,2013-07-07,0,0,Yes,1,21.905933
1,TA-00002,S-0066,2013-07-07,0,0,Yes,2,12.341214
2,TA-00002,S-0066,2013-07-07,0,0,Yes,5,6.601826
3,TA-00002,S-0066,2013-07-07,0,0,Yes,10,4.687770
4,TA-00002,S-0066,2013-07-07,0,0,Yes,25,3.541561


In [14]:
bill_of_materials.head()

,tube_assembly_id,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8
0,TA-00001,C-1622,2.0,C-1629,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00003,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00004,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00005,C-1624,1.0,C-1631,1.0,C-1641,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
train.head()

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,cost
0,TA-00002,S-0066,2013-07-07,0,0,Yes,1,21.905933
1,TA-00002,S-0066,2013-07-07,0,0,Yes,2,12.341214
2,TA-00002,S-0066,2013-07-07,0,0,Yes,5,6.601826
3,TA-00002,S-0066,2013-07-07,0,0,Yes,10,4.687770
4,TA-00002,S-0066,2013-07-07,0,0,Yes,25,3.541561


In [16]:
test.head()

,id,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity
0,1,TA-00001,S-0066,2013-06-23,0,0,Yes,1
1,2,TA-00001,S-0066,2013-06-23,0,0,Yes,2
2,3,TA-00001,S-0066,2013-06-23,0,0,Yes,5
3,4,TA-00001,S-0066,2013-06-23,0,0,Yes,10
4,5,TA-00001,S-0066,2013-06-23,0,0,Yes,25


In [17]:
tube.head()

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,TA-00001,SP-0035,12.70,1.65,164.0,5,38.10,N,N,N,N,EF-003,EF-003,0,0,0
1,TA-00002,SP-0019,6.35,0.71,137.0,8,19.05,N,N,N,N,EF-008,EF-008,0,0,0
2,TA-00003,SP-0019,6.35,0.71,127.0,7,19.05,N,N,N,N,EF-008,EF-008,0,0,0
3,TA-00004,SP-0019,6.35,0.71,137.0,9,19.05,N,N,N,N,EF-008,EF-008,0,0,0
4,TA-00005,SP-0029,19.05,1.24,109.0,4,50.80,N,N,N,N,EF-003,EF-003,0,0,0


In [19]:
train.merge(bill_of_materials, on='tube_assembly_id')

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,cost,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8
0,TA-00002,S-0066,2013-07-07,0,0,Yes,1,21.905933,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,S-0066,2013-07-07,0,0,Yes,2,12.341214,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00002,S-0066,2013-07-07,0,0,Yes,5,6.601826,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00002,S-0066,2013-07-07,0,0,Yes,10,4.687770,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00002,S-0066,2013-07-07,0,0,Yes,25,3.541561,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TA-00002,S-0066,2013-07-07,0,0,Yes,50,3.224406,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TA-00002,S-0066,2013-07-07,0,0,Yes,100,3.082521,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,TA-00002,S-0066,2013-07-07,0,0,Yes,250,2.999060,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,TA-00004,S-0066,2013-07-07,0,0,Yes,1,21.972702,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TA-00004,S-0066,2013-07-07,0,0,Yes,2,12.407983,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
train.supplier.unique()

array(['S-0066', 'S-0026', 'S-0030', 'S-0013', 'S-0062', 'S-0014',
       'S-0072', 'S-0064', 'S-0041', 'S-0054', 'S-0087', 'S-0031',
       'S-0015', 'S-0081', 'S-0018', 'S-0005', 'S-0108', 'S-0058',
       'S-0109', 'S-0012', 'S-0096', 'S-0097', 'S-0070', 'S-0090',
       'S-0025', 'S-0074', 'S-0024', 'S-0080', 'S-0029', 'S-0008',
       'S-0023', 'S-0004', 'S-0043', 'S-0022', 'S-0027', 'S-0051',
       'S-0104', 'S-0092', 'S-0007', 'S-0011', 'S-0105', 'S-0111',
       'S-0061', 'S-0095', 'S-0059', 'S-0078', 'S-0068', 'S-0060',
       'S-0046', 'S-0056', 'S-0106', 'S-0042', 'S-0006', 'S-0003',
       'S-0009', 'S-0107', 'S-0050'], dtype=object)

In [ ]:
train['quote_date'] = pd.to_datetime(train['quote_date'], infer_datetime_format=True)
test['quote_date'] = pd.to_datetime(test['quote_date'], infer_datetime_format=True)

In [23]:
train['quote_date'].describe()

count                   30213
unique                   1781
top       2013-10-01 00:00:00
freq                     2877
first     1982-09-22 00:00:00
last      2017-01-01 00:00:00
Name: quote_date, dtype: object

In [24]:
train_tube_assemblies = train['tube_assembly_id'].unique()
test_tube_assemblies = test['tube_assembly_id'].unique()
len(train_tube_assemblies), len(test_tube_assemblies)

(8855, 8856)

In [ ]:
from sklearn.model_selection import train_test_split
traing_tube_assemblies, val_tube_assemblies = train_test_split(
    train_tube_assemblies, random_state=42)

In [26]:
len(train_tube_assemblies), len(val_tube_assemblies)


(8855, 2214)

In [34]:
X_train = train[train.tube_assembly_id.isin(train_tube_assemblies)]
X_val = train[train.tube_assembly_id.isin(val_tube_assemblies)]
train.shape, val.shape, train.shape

((30213, 8), (7585, 8), (30213, 8))

In [ ]:
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [36]:
train.select_dtypes('object')

,tube_assembly_id,supplier,bracket_pricing
0,TA-00002,S-0066,Yes
1,TA-00002,S-0066,Yes
2,TA-00002,S-0066,Yes
3,TA-00002,S-0066,Yes
4,TA-00002,S-0066,Yes
5,TA-00002,S-0066,Yes
6,TA-00002,S-0066,Yes
7,TA-00002,S-0066,Yes
8,TA-00004,S-0066,Yes
9,TA-00004,S-0066,Yes


In [37]:
train.bracket_pricing.replace({'Yes':1, "No":0})


0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
30183    0
30184    1
30185    1
30186    0
30187    0
30188    0
30189    0
30190    0
30191    0
30192    0
30193    1
30194    1
30195    1
30196    1
30197    1
30198    0
30199    1
30200    1
30201    0
30202    1
30203    1
30204    1
30205    1
30206    1
30207    1
30208    0
30209    0
30210    0
30211    0
30212    0
Name: bracket_pricing, Length: 30213, dtype: int64

In [38]:
target = 'cost'
y_train = train[target]
y_val = val[target]
y_pred = np.full_like(y_val, fill_value=y_train.mean())
print('Validation RMSLE, Mean Baseline:', rmsle(y_val, y_pred))

Validation RMSLE, Mean Baseline: 0.9412105708309508


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import category_encoders as ce


In [48]:
categorical_features = ['tube_assembly_id','supplier'
    
]

numeric_features = X_train.select_dtypes('number').columns.tolist()
features = categorical_features + numeric_features

X_train = train[features]
X_val = val[features]
X_train.drop('cost', axis=1)


,tube_assembly_id,supplier,annual_usage,min_order_quantity,quantity
0,TA-00002,S-0066,0,0,1
1,TA-00002,S-0066,0,0,2
2,TA-00002,S-0066,0,0,5
3,TA-00002,S-0066,0,0,10
4,TA-00002,S-0066,0,0,25
5,TA-00002,S-0066,0,0,50
6,TA-00002,S-0066,0,0,100
7,TA-00002,S-0066,0,0,250
8,TA-00004,S-0066,0,0,1
9,TA-00004,S-0066,0,0,2


In [ ]:
# Make pipeline!
pipeline = make_pipeline(
    ce.OneHotEncoder(),
    SimpleImputer(), 
    RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    
)

# Fit on train, score on val, predict on test
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))
y_pred = pipeline.predict(X_test)

RMSLE SCORE

In [47]:
y_pred = pipeline.predict(X_val)
print(f'Validation RMSLE, Random Forest with {features}')
print(rmsle(y_val, y_pred))

Validation RMSLE, Random Forest with ['tube_assembly_id', 'supplier', 'annual_usage', 'min_order_quantity', 'quantity', 'cost']
0.0007168810723283815


RMSE SCORE

In [ ]:

r2 = model.score(X_val, y_val)
print('Validation R^2', r2)

In [ ]:
%matplotlib inline
sns.distplot(y_train)


In [ ]:
sns.distplot(y_val);

In [ ]:
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)
y_pred_log = np.log1p(y_pred)

sns.distplot(y_train_log);

In [ ]:
y_val_log = np.log1p(y_val)
y_pred_log = np.log1p(y_pred)
sns.distplot(y_val_log, label='True')
sns.distplot(y_pred_log, label='Predicted')
plt.legend();